In [1]:
import yaml
from modules.agent import OpenAICommentAnalysisAgent


with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')


In [ ]:
agent.comment_analyze('This desk is sturdy!')

In [ ]:
resp = agent.points_extract("I spent up on the desktop but the slab of wood was beautiful (and heavy). The legs and other materials appear to be all metal. Wood + metal is rare these days everything is plastic.   Have only owned it for a week but love it so far. Plan to keep for the rest of my life.")
print(resp)

In [ ]:
import pandas as pd


df = pd.read_csv('./docs/testset_amz_bsr_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

# Scratchs

In [ ]:
import re
import yaml
import pandas as pd
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


df = pd.read_csv('./docs/testset_stations_bs_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')

for i in tqdm(range(len(df))):
    comment = df.iloc[i]['content']
    if not isinstance(comment, str):
        continue

    points = agent.points_extract(comment)
    # print(points)
    df.loc[i, '体验点分析'] = points
    extract_points = extract_output(points)
    df.loc[i, '体验点'] = ''.join(extract_points)
    # print(extract_points[0])
    # break

df.to_csv('./results/stations_bs_reviews_points.csv', sep='\t')

In [2]:
from tools.lilfunc import read_xlsx


df = read_xlsx('buffer/Products(32)-US-20250401.xlsx')
print(df.columns)
print(df.shape)

Index(['#', '图片', 'ASIN', 'SKU', '详细参数', '品牌', '品牌链接', '商品标题', '产品卖点',
       '商品详情页链接', '商品主图', '父ASIN', '类目路径', '大类目', '大类BSR', '大类BSR增长数',
       '大类BSR增长率', '小类目', '小类BSR', '月销量', '月销量增长率', '月销售额($)', '子体销量',
       '子体销售额($)', '变体数', '价格($)', 'Prime价格($)', 'Coupon', 'Q&A数', '评分数',
       '月新增\n评分数', '评分', '留评率', 'FBA($)', '毛利率', '评级', '上架时间', '上架天数', '配送方式',
       'LQS', '卖家数', 'Buybox卖家', 'BuyBox类型', '卖家所属地', '卖家信息', '卖家首页',
       'Best Seller标识', 'Amazon's Choice', 'New Release标识', 'A+页面', '视频介绍',
       'SP广告', '品牌故事', '品牌广告', '7天促销', 'AC关键词', '重量', '重量.1', '体积', '体积.1',
       '包装重量', '包装体积', '近1年销量', '近一年销售额($)', '平均价格($)', '市场份额', '卖家',
       '卖家精灵官网：https://www.sellersprite.com'],
      dtype='object')
(42, 68)


In [3]:
import os
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_pdf_path, output_folder, num_splits):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 打开原始PDF文件
    with open(input_pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        
        # 计算每个分割文件的页数
        pages_per_file = num_pages // num_splits
        
        # 循环创建分割的PDF文件
        for i in range(num_splits):
            # 创建一个PDF写入对象
            writer = PdfWriter()
            
            # 添加页面到写入对象
            start_page = i * pages_per_file
            end_page = start_page + pages_per_file
            if i == num_splits - 1:  # 最后一个分割可能包含更多的页面
                end_page = num_pages
            for page_num in range(start_page, end_page):
                writer.add_page(reader.pages[page_num])
            
            # 输出文件名
            output_filename = f"output_{i+1}.pdf"
            output_path = os.path.join(output_folder, output_filename)
            
            # 写入文件
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)
            print(f"Created {output_path}")

# 使用示例
input_pdf_path = 'a.pdf'  # 替换为你的PDF文件路径
output_folder = 'split_pdfs'  # 输出文件夹路径
num_splits = 8  # 分割成6份

split_pdf(input_pdf_path, output_folder, num_splits)

Created split_pdfs/output_1.pdf
Created split_pdfs/output_2.pdf
Created split_pdfs/output_3.pdf
Created split_pdfs/output_4.pdf
Created split_pdfs/output_5.pdf
Created split_pdfs/output_6.pdf
Created split_pdfs/output_7.pdf
Created split_pdfs/output_8.pdf


3626.4